In [1]:
%config IPCompleted.greedy = True

# Import Libraries

In [2]:
import os
import json
from azureml.core import Environment, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.image import ContainerImage
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from sklearn.metrics import accuracy_score, confusion_matrix
from pathlib import Path

# Get Workspace

In [3]:
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourceGroups/disasterLocator/providers/Microsoft.MachineLearningServices/workspaces/disasterLocatorML',
 'name': 'disasterLocatorML',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '90beb71e-3575-4cbd-af2a-cb2c303edc7b',
 'description': '',
 'friendlyName': 'disasterLocatorML',
 'creationTime': '2020-11-11T14:39:30.8601986+00:00',
 'containerRegistry': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourceGroups/disasterLocator/providers/Microsoft.ContainerRegistry/registries/90beb71e35754cbdaf2acb2c303edc7b',
 'keyVault': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourcegroups/disasterlocator/providers/microsoft.keyvault/vaults/disasterlocato9806975694',
 'applicationInsights': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourcegroups/disasterlocator/providers/microsoft.insights/components/disasterlocato5776021600',
 

# Define Environment

In [4]:
keras_env = Environment.from_conda_specification(name='keras-env', file_path='../conda_dependencies.yml')

# Create Configuration

In [5]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Tweets",  "method" : "keras"}, 
                                               description='Predict Disaster from Tweets with keras')

# Define an inference configuration

In [6]:
script_folder = os.path.join(Path(os.getcwd()).resolve().parent, 'src')
inference_config = InferenceConfig(source_directory=script_folder, entry_script='scoring.py', environment=keras_env)

# Retrieve the Model

In [7]:
model = Model(ws, 'disaster_predictor_cnn')

# Deploy Model

In [8]:
service = Model.deploy(
            workspace = ws,
            name = "disaster-predictor-cnn",
            models = [model],
            inference_config = inference_config,
            deployment_config = deployment_config,
            overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [10]:
service.scoring_uri

'http://81e09d9c-2563-4a38-be07-45610754431e.eastus.azurecontainer.io/score'

# Test deployed Service

In [11]:
from azureml.core import Dataset

# get data
dataset = Dataset.get_by_name(ws, name='disaster_tweets_train')
tweets_pd = dataset.to_pandas_dataframe()
tweets_pd.dropna(inplace=True)

# test with n samples
n = 30
tweets_sample = tweets_pd.sample(int(len(tweets_pd)*0.4))
X_test = tweets_pd.sample(30)['text'].values
y_test = tweets_pd.sample(30)['target'].values

test_samples = json.dumps({"data": X_test.tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict
results = service.run(input_data=test_samples)
results = [0 if result < 0.5 else 1 for result in results]

In [12]:
accuracy_score(y_test, results)

0.36666666666666664

In [13]:
confusion_matrix(y_test, results)

array([[ 7,  6],
       [13,  4]], dtype=int64)

# Test with Custom Sample

In [14]:
custom_sample = json.dumps({"data": ["Flooding in Arizona"]})
custom_sample = bytes(custom_sample, encoding='utf8')
custom_result = service.run(input_data=custom_sample)
if custom_result[0]==1:
    print("disaster!")
else:
    print("no disaster")

no disaster


In [17]:
custom_sample = json.dumps({"data": ["20 dead after Earthquake"]})
custom_sample = bytes(custom_sample, encoding='utf8')
custom_result = service.run(input_data=custom_sample)
if custom_result[0]==1:
    print("disaster!")
else:
    print("no disaster")

no disaster
